In [ ]:
from functools import partial
from tensorflow import keras
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models

In [ ]:
def preprocess(img, label):
    img = tf.cast(img, tf.float32)  # Typ sicherstellen
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = tf.image.grayscale_to_rgb(img)
    return img, label

In [19]:
DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")
model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

/home/tibor/tf/tf-gpu/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
model = keras.applications.resnet50.ResNet50(weights="imagenet")

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


In [17]:
mnist = tfds.load(name="mnist", batch_size=32, as_supervised=True)
mnist_train, mnist_test = mnist["train"].repeat().prefetch(1), mnist["test"]

In [20]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
model.fit(mnist_train, steps_per_epoch=60000 // 32, epochs=5)

Epoch 1/5


2025-05-06 16:56:13.394847: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
I0000 00:00:1746543373.401722    1776 service.cc:152] XLA service 0x7f4fb00044f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746543373.401897    1776 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2025-05-06 16:56:13.436341: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746543373.553675    1776 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-06 16:56:14.248124: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1225', 24 bytes spill stores, 24 bytes spill loads

202

  57/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 9.1411 

I0000 00:00:1746543376.638145    1776 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 1.5166
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.1852
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.1273
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0954
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0777


In [22]:
y_pred = model.predict(mnist_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [26]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]
    
    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [31]:
model_res = keras.models.Sequential()
model_res.add(DefaultConv2D(64, kernel_size=7, strides=2,
                            input_shape=[28, 28, 1]))
model_res.add(keras.layers.BatchNormalization())
model_res.add(keras.layers.Activation("relu"))
model_res.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model_res.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model_res.add(keras.layers.GlobalAvgPool2D())
model_res.add(keras.layers.Flatten())
model_res.add(keras.layers.Dense(10, activation="softmax"))

model_res.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
model_res.fit(mnist_train, steps_per_epoch=60000 // 32, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.4505
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - loss: 0.0482
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - loss: 0.0227
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - loss: 0.0122
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - loss: 0.0081


In [41]:
dataset = tfds.load(name="mnist")
mnist_test = dataset["test"]

mnist_test = mnist_test.repeat(5).batch(32).prefetch(1)
for item in mnist_test:
    images = item["image"]
    labels = item["label"]

In [42]:
y_pred = model_res.predict(images)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [47]:
metrics(labels, y_pred)

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
# x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)

BATCH_SIZE = 32
IMG_SIZE = 224


train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


base_model = tf.keras.applications.EfficientNetV2B0(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = True

model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, epochs=5, validation_data=test_ds)

Epoch 1/5


2025-05-15 10:27:16.679256: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_23702', 204 bytes spill stores, 204 bytes spill loads

2025-05-15 10:27:16.703987: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_23702_0', 464 bytes spill stores, 1372 bytes spill loads

2025-05-15 10:27:16.996346: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_23709', 4 bytes spill stores, 4 bytes spill loads

2025-05-15 10:27:17.024790: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_23702', 5616 bytes spill stores, 5612 bytes spill loads

2025-05-15 10:27:17.158003:

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9501 - loss: 0.1676

2025-05-15 10:28:52.694889: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2300_0', 200 bytes spill stores, 536 bytes spill loads

2025-05-15 10:28:52.755695: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2300', 116 bytes spill stores, 116 bytes spill loads

2025-05-15 10:28:52.875488: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2300', 4 bytes spill stores, 4 bytes spill loads

2025-05-15 10:28:53.139079: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2300', 5616 bytes spill stores, 5612 bytes spill loads

2025-05-15 10:28:53.339691: I ex

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 122s 45ms/step - accuracy: 0.9501 - loss: 0.1676 - val_accuracy: 0.7771 - val_loss: 0.6877
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 44ms/step - accuracy: 0.9863 - loss: 0.0448 - val_accuracy: 0.0980 - val_loss: 5.8501
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 43ms/step - accuracy: 0.9898 - loss: 0.0345 - val_accuracy: 0.8659 - val_loss: 0.3845
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 42ms/step - accuracy: 0.9907 - loss: 0.0288 - val_accuracy: 0.1135 - val_loss: 10.1587
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 43ms/step - accuracy: 0.9925 - loss: 0.0251 - val_accuracy: 0.9872 - val_loss: 0.0425
